In [1]:
#import packages
from os import listdir
from nipype.interfaces.io import DataSink, SelectFiles, DataGrabber # Data i/o
from nipype.interfaces.utility import IdentityInterface, Function     # utility
from nipype.pipeline.engine import Node, Workflow, JoinNode        # pypeline engine

from nipype.interfaces.fsl.model import Randomise, GLM, Cluster
from nipype.interfaces.freesurfer.model import Binarize
from nipype.interfaces.fsl.utils import ImageMeants, Merge, Split
from nipype.interfaces.fsl.maths import ApplyMask

#set output file type for FSL to NIFTI
from nipype.interfaces.fsl.preprocess import FSLCommand
FSLCommand.set_default_output_type('NIFTI')

# MATLAB setup - Specify path to current SPM and the MATLAB's default mode
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('~/spm12')
MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")

# Set study variables
studyhome = '/Users/catcamacho/Box/BABIES'
raw_data = studyhome + '/raw'
preproc_dir = studyhome + '/processed/preproc'
output_dir = studyhome + '/processed/analysis2'
workflow_dir = studyhome + '/workflows'
roi_dir = studyhome + '/ROIs'
group_con = studyhome + '/misc/tcon.con'
group_mat = studyhome + '/misc/design.mat'
proc_cores = 2

#subjects_list = ['021-BABIES-T1','033x-BABIES-T1'] #listdir(raw_data)
subjects_list = open(studyhome + '/misc/subjects.txt').read().splitlines()

template_brain = studyhome + '/templates/T2w_BABIES_template_2mm.nii'

# ROIs for connectivity analysis
Lamyg = roi_dir + '/L_amyg_4mm.nii'
Ramyg = roi_dir + '/R_amyg_4mm.nii'

ROIs = [Lamyg, Ramyg]
rois = ['L_amyg','R_amyg']

min_clust_size = 25

In [2]:
## File handling
# Identity node- select subjects
infosource = Node(IdentityInterface(fields=['subject_id','ROIs']),
                     name='infosource')
infosource.iterables = [('subject_id', subjects_list),('ROIs',ROIs)]


# Data grabber- select fMRI and ROIs
templates = {'orig_func': preproc_dir + '/smoothed_filt_func/_subject_id_{subject_id}/func_filtered_smooth.nii'}
selectfiles = Node(SelectFiles(templates), name='selectfiles')

# Datasink- where our select outputs will go
datasink = Node(DataSink(), name='datasink')
datasink.inputs.base_directory = output_dir
datasink.inputs.container = output_dir
substitutions = [('_subject_id_', ''),
                ('_ROIs_..Users..catcamacho..Box..BABIES..ROIs..','')]
datasink.inputs.substitutions = substitutions

In [3]:
## Seed-based level 1

# Extract ROI timeseries
ROI_timeseries = Node(ImageMeants(), name='ROI_timeseries', iterfield='mask')

def txt2vest(timeseries):
    from os import system, path
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    
    vest = 'vest.mat'
    system('Text2Vest '+ timeseries + ' ' + vest)
    vest_file = path.abspath(vest)
    return(vest_file)

txt2vest = Node(name='txt2vest', 
                interface=Function(input_names=['timeseries'], 
                                   outputs_names=['vest_file'],
                                   function=txt2vest))

# model ROI connectivity
glm = Node(GLM(out_file='betas.nii',out_cope='cope.nii'), name='glm', iterfield='design')

In [4]:
sbc1_workflow = Workflow(name='sbc1_workflow')
sbc1_workflow.connect([(infosource,selectfiles,[('subject_id','subject_id')]),
                       (selectfiles,ROI_timeseries,[('orig_func','in_file')]),
                       (infosource,ROI_timeseries,[('ROIs','mask')]),
                       (ROI_timeseries,glm,[('out_file','design')]),
                       #(ROI_timeseries,txt2vest,[('out_file','timeseries')]),
                       #(txt2vest,glm,[('vest_file','design')]),
                       (selectfiles,glm,[('orig_func','in_file')]),
                       (ROI_timeseries, datasink, [('out_file','roi_ts')]),
                       (glm,datasink,[('out_cope','glm_seed_copes')]),
                       (glm,datasink,[('out_file','glm_betas')])
                      ])
sbc1_workflow.base_dir = workflow_dir
sbc1_workflow.write_graph(graph2use='flat')
sbc1_workflow.run('MultiProc', plugin_args={'n_procs': proc_cores})

170731-12:49:48,657 workflow INFO:
	 Generated workflow graph: /Users/catcamacho/Box/BABIES/workflows/sbc1_workflow/graph.dot.png (graph2use=flat, simple_form=True).
170731-12:49:48,928 workflow INFO:
	 Workflow sbc1_workflow settings: ['check', 'execution', 'logging']
170731-12:49:49,57 workflow INFO:
	 Running in parallel.
170731-12:49:49,63 workflow INFO:
	 Executing: selectfiles.a22 ID: 0
170731-12:49:49,66 workflow INFO:
	 Executing: selectfiles.a11 ID: 1
170731-12:49:49,70 workflow INFO:
	 Executing node selectfiles.a22 in dir: /Users/catcamacho/Box/BABIES/workflows/sbc1_workflow/_ROIs_..Users..catcamacho..Box..BABIES..ROIs..R_amyg_4mm.nii_subject_id_032-BABIES-T1/selectfiles
170731-12:49:49,72 workflow INFO:
	 Executing node selectfiles.a11 in dir: /Users/catcamacho/Box/BABIES/workflows/sbc1_workflow/_ROIs_..Users..catcamacho..Box..BABIES..ROIs..L_amyg_4mm.nii_subject_id_036-BABIES-T1/selectfiles
170731-12:49:49,141 workflow INFO:
	 [Job finished] jobname: selectfiles.a22 jobid:

In [5]:
infosource2 = Node(IdentityInterface(fields=['roi']),
                   name='infosource2')
infosource2.iterables = ('roi',rois)


# Data grabber- select fMRI and ROIs
templates = {'roi': 'glm_seed_copes/%s_4mm.nii*/cope.nii'}

datagrabber = Node(DataGrabber(infields=['roi'], 
                               outfields=['roi'],
                               sort_filelist=True,
                               base_directory=output_dir,
                               template='glm_seed_copes/%s_4mm.nii*/cope.nii',
                               field_template=templates,
                               template_args=dict(roi=[['roi']])),
                   name='datagrabber')


In [6]:
## Level 2

# merge param estimates across all subjects per seed
merge = Node(Merge(dimension='t'),
             name='merge')

# FSL randomise for higher level analysis
highermodel = Node(Randomise(tfce=True,
                             raw_stats_imgs= True,
                             design_mat=group_mat,
                             tcon=group_con),
                   name = 'highermodel')

## Cluster results

# make binary masks of sig clusters
binarize = Node(Binarize(min=0.95, max=1.0), 
                name='binarize', 
                iterfield='in_file')

# mask T-map before clustering
mask_tmaps = Node(ApplyMask(), name='mask_tmaps')

# clusterize and extract cluster stats/peaks
clusterize = Node(Cluster(threshold=3.5, 
                          out_index_file='outindex.nii', 
                          out_localmax_txt_file='localmax.txt'), 
                  name='clusterize')

# make pictures if time

In [7]:
sbc2_workflow = Workflow(name='sbc2_workflow')
sbc2_workflow.connect([(infosource2,datagrabber,[('roi','roi')]),
                       (datagrabber,merge,[('roi','in_files')]),
                       (merge,highermodel,[('merged_file','in_file')]),

                       (highermodel,datasink,[('t_corrected_p_files','rand_corrp_files')]),
                       (highermodel,datasink,[('tstat_files','rand_tstat_files')])
                      ])
sbc2_workflow.base_dir = workflow_dir
sbc2_workflow.write_graph(graph2use='flat')
sbc2_workflow.run('MultiProc', plugin_args={'n_procs': proc_cores})

170731-12:56:10,246 workflow INFO:
	 Generated workflow graph: /Users/catcamacho/Box/BABIES/workflows/sbc2_workflow/graph.dot.png (graph2use=flat, simple_form=True).
170731-12:56:10,272 workflow INFO:
	 Workflow sbc2_workflow settings: ['check', 'execution', 'logging']
170731-12:56:10,292 workflow INFO:
	 Running in parallel.
170731-12:56:10,295 workflow INFO:
	 Executing: datagrabber.a0 ID: 0
170731-12:56:10,298 workflow INFO:
	 Executing: datagrabber.a1 ID: 4
170731-12:56:10,302 workflow INFO:
	 Executing node datagrabber.a0 in dir: /Users/catcamacho/Box/BABIES/workflows/sbc2_workflow/_roi_L_amyg/datagrabber
170731-12:56:10,304 workflow INFO:
	 Executing node datagrabber.a1 in dir: /Users/catcamacho/Box/BABIES/workflows/sbc2_workflow/_roi_R_amyg/datagrabber
170731-12:56:10,335 workflow INFO:
	 [Job finished] jobname: datagrabber.a0 jobid: 0
170731-12:56:10,338 workflow INFO:
	 Executing: merge.a0 ID: 1
170731-12:56:10,345 workflow INFO:
	 [Job finished] jobname: datagrabber.a1 jobid:

In [8]:
# Identity node- select subjects
infosource3 = Node(IdentityInterface(fields=['roi']),
                   name='infosource3')
infosource3.iterables = [('roi',rois)]


# Data grabber- select fMRI and ROIs
templates = {'pcorrT': output_dir + '/rand_corrp_files/_roi_{roi}/tbss__tfce_corrp_tstat1.nii', 
             'tstat': output_dir + '/rand_tstat_files/_roi_{roi}/tbss__tstat1.nii'}
selectfiles2 = Node(SelectFiles(templates), name='selectfiles2')

In [9]:
sbc3_workflow = Workflow(name='sbc3_workflow')
sbc3_workflow.connect([(infosource3,selectfiles2, [('roi','roi')]),
                       (selectfiles2, binarize, [('pcorrT','in_file')]),
                       (binarize, mask_tmaps, [('binary_file','mask_file')]),
                       (selectfiles2, mask_tmaps, [('tstat','in_file')]),
                       (mask_tmaps, clusterize, [('out_file','in_file')]),
                       
                       (binarize,datasink,[('binary_file','binary_pval')]),
                       (mask_tmaps,datasink,[('out_file','masked_tmaps')]),
                       (clusterize,datasink,[('index_file','cluster_index_file')]),
                       (clusterize,datasink,[('localmax_txt_file','localmax_txt_file')])
                      ])
sbc3_workflow.base_dir = workflow_dir
sbc3_workflow.write_graph(graph2use='flat')
sbc3_workflow.run('MultiProc', plugin_args={'n_procs': proc_cores})

170731-13:06:01,891 workflow INFO:
	 Generated workflow graph: /Users/catcamacho/Box/BABIES/workflows/sbc3_workflow/graph.dot.png (graph2use=flat, simple_form=True).
170731-13:06:01,921 workflow INFO:
	 Workflow sbc3_workflow settings: ['check', 'execution', 'logging']
170731-13:06:01,948 workflow INFO:
	 Running in parallel.
170731-13:06:01,952 workflow INFO:
	 Executing: selectfiles2.a1 ID: 0
170731-13:06:01,955 workflow INFO:
	 Executing: selectfiles2.a0 ID: 3
170731-13:06:01,959 workflow INFO:
	 Executing node selectfiles2.a1 in dir: /Users/catcamacho/Box/BABIES/workflows/sbc3_workflow/_roi_R_amyg/selectfiles2
170731-13:06:01,961 workflow INFO:
	 Executing node selectfiles2.a0 in dir: /Users/catcamacho/Box/BABIES/workflows/sbc3_workflow/_roi_L_amyg/selectfiles2
170731-13:06:02,24 workflow INFO:
	 [Job finished] jobname: selectfiles2.a1 jobid: 0
170731-13:06:02,27 workflow INFO:
	 [Job finished] jobname: selectfiles2.a0 jobid: 3
170731-13:06:02,32 workflow INFO:
	 Executing: binariz

In [10]:
# Identity node- select subjects
infosource4 = Node(IdentityInterface(fields=['roi']),
                   name='infosource4')
infosource4.iterables = [('roi',rois)]


# Data grabber- select fMRI and ROIs
templates = {'clusters': output_dir + '/cluster_index_file/_roi_{roi}/outindex.nii', 
             'cluster_table': output_dir + '/localmax_txt_file/_roi_{roi}/cluster_table.txt'}
selectfiles3 = Node(SelectFiles(templates), name='selectfiles3')

# Grab betas for stuff
templates2 = {'roi': 'glm_betas/%s_4mm.nii*-BABIES-T1/betas.nii'}

betagrabber = Node(DataGrabber(infields=['roi'], 
                               outfields=['roi'],
                               sort_filelist=True,
                               base_directory=output_dir,
                               template='glm_betas/%s_4mm.nii*-BABIES-T1/betas.nii',
                               field_template=templates2,
                               template_args=dict(roi=[['roi']])),
                   name='betagrabber')

In [11]:
# parse clusters table
def determine_clusters(clusters_table, min_clust_size):
    from os import path
    from numpy import genfromtxt
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    
    clusters = genfromtxt(clusters_table, delimiter='\t', dtype=None, skip_header=1)
    clusters_to_extract = []
    
    for t in clusters:
        if clusters[t][1] >= min_clust_size:
            clusters_to_extract.append(clusters[t][0])
    
    
    return(cluster_index)

det_clust = Node(name='det_clust', 
                 interface=Function(input_names=['clusters_table','min_clust_size'],
                                    output_names=['cluster_index'], 
                                    function=determine_clusters))
det_clust.inputs.min_clust_size=min_clust_size

# separate cluster volumes
split_clusters = Node(Split(dimension='t'), name='split_clusters')

# merge betas together
merge_betas = Node(Merge(dimension='t'), name='merge_betas')

# extract betas for each subject/roi clusters and put in table as fisher's Z scores
def extract_fisherZ(subj_betas, clusters, cluster_table):
    from os import path
    from numpy import genfromtxt, savetxt
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    
    header = []
    clusters = genfromtxt(clusters_table, delimiter='\t', dtype=None, skip_header=1)
    
    savetxt(noise_file, noise_matrix, delimiter='\t', header=header)

    return(table_path)

extract_fisherZ = Node(name='extract_fisherZ', 
                       interface=Function(input_names=['subj_betas','clusters','cluster_table'],
                                          output_names=['table_path'], 
                                          function=extract_fisherZ))

In [12]:
sbc4_workflow = Workflow(name='sbc4_workflow')
sbc4_workflow.connect([(infosource4, selectfiles3, [('roi','roi')]),
                       (infosource4, betagrabber, [('roi','roi')]),
                       (selectfiles3, split_clusters, [('clusters','in_file')]),
                       (betagrabber, merge_betas, [('roi','in_files')]),
                       
                       (merge_betas, datasink, [('merged_file','merged_betas')]),
                       (split_clusters, datasink, [('out_files','split_clusters')])
                      ])
sbc4_workflow.base_dir = workflow_dir
sbc4_workflow.write_graph(graph2use='flat')
sbc4_workflow.run('MultiProc', plugin_args={'n_procs': proc_cores})

170729-15:43:06,41 workflow INFO:
	 Generated workflow graph: /Users/catcamacho/Box/BABIES/workflows/sbc4_workflow/graph.dot.png (graph2use=flat, simple_form=True).
170729-15:43:06,67 workflow INFO:
	 Workflow sbc4_workflow settings: ['check', 'execution', 'logging']
170729-15:43:06,86 workflow INFO:
	 Running in parallel.
170729-15:43:06,91 workflow INFO:
	 Executing: betagrabber.a1 ID: 0
170729-15:43:06,95 workflow INFO:
	 Executing: selectfiles3.a1 ID: 1
170729-15:43:06,99 workflow INFO:
	 Executing node betagrabber.a1 in dir: /Users/catcamacho/Box/BABIES/workflows/sbc4_workflow/_roi_R_amyg/betagrabber
170729-15:43:06,102 workflow INFO:
	 Executing node selectfiles3.a1 in dir: /Users/catcamacho/Box/BABIES/workflows/sbc4_workflow/_roi_R_amyg/selectfiles3
170729-15:43:06,168 workflow INFO:
	 [Job finished] jobname: betagrabber.a1 jobid: 0
170729-15:43:06,170 workflow ERROR:
	 [u'Node selectfiles3.a1 failed to run on host Cats-MacBook-Pro-2.local.']
170729-15:43:06,171 workflow INFO:
	

RuntimeError: Workflow did not execute cleanly. Check log for details